# Notebook 01: Data Cleaning

This notebook cleans/merges the raw datasets into a combined dataset.

Raw Datasets:
- WATCH FLOW CHART.xlsx : contains XYZ's records of each model's sales for the brand since 2015 (when they started distributing the brand).
- PROPOSAL.xlsx : contain's the brand's current collection and XYZ's order on April 16, 2022. It contains older models found in the flow chart and new models with no previous sales.
- ALL ITEMS FROM WEBSITE.xlsx : contains all watches from the brand's website, and includes more attributes. 
- ITEMS CREATED 2022H3.xlsx : contains the new (2022H3) watch attributes from the brand's website.
- Plan Order 2022H3.xlsx : contains XYZ's planned order for the 2022H3 watches.

Cleaned Dataset:
- df_models.csv : cleaned and merged version of all the data in the raw datasets

## Imports

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import warnings

### Custom Functions

In [2]:
from custom_functions import *

## Data Cleaning

### Watch Flow Chart

This dataset contains XYZ's records of each model's sales for the brand since 2015 (when they started distributing the brand).

In [3]:
df_watch_chart = pd.read_excel('../data/raw_datasets/WATCH FLOW CHART.xlsx',header=1)

# Standardize column names
snake_case(df_watch_chart)
replace_months_from_df(df_watch_chart)

# Select desired columns
df_watch_chart = df_watch_chart[['style','retail','collection', 'size',
       'gender', 'status', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022-01', '2022-02', '2022-03', '2022-04',
       '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10',
       '2022-11', '2022-12']]

# Rename columns
df_watch_chart.rename({'style':'style_id','retail':'retail_price','size':'case','so':'qty_sales_order','oh':'qty_on_hand'},axis='columns',inplace=True)

# Drop null row (not needed)
df_watch_chart.dropna(subset='style_id', inplace=True)

# Manual Process - Identify and deal with null values
# Retail price (1 model which was discontinued and never had sales)
df_watch_chart.dropna(subset='retail_price', inplace=True)
# Collection
df_watch_chart.loc[646,'collection'] = 'FORRESTER CHRONO'
df_watch_chart.loc[786,'collection'] = 'JACQUELINE'
# Case
df_watch_chart.loc[646,'case'] = '46MM'
df_watch_chart.loc[786,'case'] = '36MM'
# Sales years
df_watch_chart[['2015','2016','2017','2018','2019','2020','2021','2022-01',
       '2022-02', '2022-03', '2022-04']] = df_watch_chart[['2015','2016','2017','2018','2019','2020','2021','2022-01',
       '2022-02', '2022-03', '2022-04']].fillna(0)
# Status
df_watch_chart.fillna('no status reported',inplace=True)

# Estimate monthly sales from yearly sales
yearly_to_monthly(df_watch_chart)

# Standardize values
clean_values(df_watch_chart)

# Sort columns with dates
df_watch_chart = sort_columns(df_watch_chart)

In [4]:
df_watch_chart.isna().sum().sum()

0

In [5]:
df_watch_chart.head(1)

,style_id,retail_price,collection,case,gender,status,2015-01,2015-02,2015-03,2015-04,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
1,am4141,129.0,colleague,28mm,ladies,no status reported,4.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_watch_chart.columns.values

array(['style_id', 'retail_price', 'collection', 'case', 'gender',
       'status', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05',
       '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11',
       '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05',
       '2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11',
       '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05',
       '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11',
       '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05',
       '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11',
       '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05',
       '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11',
       '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05',
       '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11',
       '2020-12', '2021-01', '2021-02', '2021-03', '2021-0

### Brand Proposal

This dataset contain's the brand's current collection and XYZ's order on April 16, 2022. It contains older models found in the flow chart and new models with no previous sales.

In [7]:
df_proposal = pd.read_excel('../data/raw_datasets/PROPOSAL.xlsx')

# Standardize column names
snake_case(df_proposal)

# Select desired columns
df_proposal = df_proposal[['material', 'status', 'retail', 'gender', 'collection',
       'case', 'inventory', 'qty_oh', 'qty_oor', 'total', 
       'order', 'sold_last_6m', 'average/mo', 
       'months/sup', 'wholesale', 'cost', 'ext_cost']]

# Drop rows with total sheet calculations
df_proposal.dropna(subset='material',inplace=True)

# Rename columns
df_proposal.rename({'material':'style_id','status':'priority','retail':'retail_price','inventory':'availability','qty_oh':'qty_on_hand',
                    'qty_oor':'qty_on_order','total':'qty_total_inv','order':'qty_sales_order',
                    'sold_last_6m':'qty_sold_last_6m','average/mo':'qty_avg/mo','months/sup':'months_of_supply',
                    'wholesale':'wholesale_price','cost':'xyz_cost','ext_cost':'qty_xyz_cost'},axis='columns',inplace=True)

# Cast numerical datatypes to float
df_proposal['qty_on_order'] = df_proposal['qty_on_order'].astype(float)
df_proposal['qty_sold_last_6m'] = df_proposal['qty_sold_last_6m'].astype(float)
df_proposal['months_of_supply'] = df_proposal['months_of_supply'].astype(float)

# Standardize values
clean_values(df_proposal)

#### Manual Process - Data Quality Checks

In [8]:
# Data quality checks
df_proposal['qty_total_dup'] = df_proposal['qty_on_hand'] + df_proposal['qty_on_order']
(df_proposal['qty_total_dup'] == df_proposal['qty_total_inv']).value_counts()
# Should all be true

True    300
dtype: int64

In [9]:
df_proposal['qty_avg/mo_dup'] = df_proposal['qty_sold_last_6m']/6
(df_proposal['qty_avg/mo_dup'] == df_proposal['qty_avg/mo']).value_counts()
# Should all be true

True    300
dtype: int64

In [10]:
df_proposal['months_of_supply_dup'] = [(df_proposal.loc[x,'qty_total_dup'] + df_proposal.loc[x,'qty_sales_order'])/df_proposal.loc[x,'qty_avg/mo_dup'] if df_proposal.loc[x,'qty_avg/mo_dup'] != 0 else np.nan for x in df_proposal.index]
(df_proposal['months_of_supply_dup'] == df_proposal['months_of_supply']).value_counts()
# Some may be False, check below is true

True     216
False     84
dtype: int64

In [11]:
# Check that the falses above are because they are null values
(df_proposal['months_of_supply_dup'].isna() == df_proposal['months_of_supply'].isna()).value_counts()
# Should all be true

True    300
dtype: int64

In [12]:
df_proposal['xyz_cost_dup'] = df_proposal['wholesale_price']*0.7
(df_proposal['xyz_cost_dup'] == df_proposal['xyz_cost']).value_counts()
# Should all be true

True    300
dtype: int64

In [13]:
# Drop all duplicated columns
df_proposal.drop(columns=['qty_total_dup','qty_avg/mo_dup','months_of_supply_dup','xyz_cost_dup'],inplace=True)

In [14]:
df_proposal.isna().sum().sum()

84

In [15]:
df_proposal.head(1)

,style_id,priority,retail_price,gender,collection,case,availability,qty_on_hand,qty_on_order,qty_total_inv,qty_sales_order,qty_sold_last_6m,qty_avg/mo,months_of_supply,wholesale_price,xyz_cost,qty_xyz_cost
0,ce1107,c,230.0,women,fb-01,35mm,available,12.0,0.0,12.0,10.0,43.0,7.166667,3.069767,115.0,80.5,805.0


In [16]:
df_proposal.columns.values

array(['style_id', 'priority', 'retail_price', 'gender', 'collection',
       'case', 'availability', 'qty_on_hand', 'qty_on_order',
       'qty_total_inv', 'qty_sales_order', 'qty_sold_last_6m',
       'qty_avg/mo', 'months_of_supply', 'wholesale_price', 'xyz_cost',
       'qty_xyz_cost'], dtype=object)

### All Watches from Brand Website

This dataset contains all watches from the brand's website, and includes more attributes.

In [17]:
df_website = pd.read_excel('../data/raw_datasets/ALL ITEMS FROM WEBSITE.xlsx')

# Standardize column names
snake_case(df_website)

# Select desired columns
df_website = df_website[['ref', 'gender', 'collection', 'size',
       'price', 'msrp', 'product_type', 'product_websites',
       'weight', 'color', 'country_of_origin', 'warranty',
       'band_color', 'band_material', 'case_material', 'clasp_type',
       'crystal_type', 'dial_color', 'movement_type', 'water_resistant',
       'max_cart_qty']] 

# Rename columns
df_website.rename({'ref':'style_id','price':'wholesale_price','msrp':'retail_price'},axis='columns',inplace=True)

# Manual process - Identify and deal with null values
# band color
df_website['band_color'] = df_website['band_color'].fillna('no color reported')
# clasp type
df_website['clasp_type'] = df_website['clasp_type'].fillna('no clasp type reported')
# movement type
df_website['movement_type'] = df_website['movement_type'].fillna('no movement type reported')
# water resistance, fill with 10000 since most likelye
df_website['max_cart_qty'] = df_website['max_cart_qty'].fillna(10000)

# Dropping the following columns because their values are all the same
df_website.drop(columns=['product_type','product_websites','weight','crystal_type'],inplace=True)

# Standardize values
clean_values(df_website)

In [18]:
df_website.isna().sum().sum()

0

In [19]:
df_website.head(1)

,style_id,gender,collection,size,wholesale_price,retail_price,color,country_of_origin,warranty,band_color,band_material,case_material,clasp_type,dial_color,movement_type,water_resistant,max_cart_qty
0,am4141,ladies,serena,28,55.0,110,mother of pearl,japan - jp,2 - year international limited warranty,silver,stainless steel,stainless steel,no clasp type reported,mother of pearl,quartz,50m - 160ft - 5atm,10000.0


In [20]:
df_website.columns.values

array(['style_id', 'gender', 'collection', 'size', 'wholesale_price',
       'retail_price', 'color', 'country_of_origin', 'warranty',
       'band_color', 'band_material', 'case_material', 'clasp_type',
       'dial_color', 'movement_type', 'water_resistant', 'max_cart_qty'],
      dtype=object)

### Items Created 2022H3

This dataset contains the new watch attributes (2022H3) from the brand's website.

In [21]:
df_new_website = pd.read_excel('../data/raw_datasets/ITEMS CREATED 2022H3.xlsx')

# Standardize column names
snake_case(df_new_website)
replace_months_from_df(df_new_website)

# Select desired columns
df_new_website = df_new_website[['item', 'collection', 'season', 'rtl',
       'gender', 'case_size', 'case_material', 'dial_color',
       'band_color', 'band_material', 'clasp', 'movement',
       'water_resistance', 'country_of_origin', 'warranty']]

# Rename columns
df_new_website.rename({'item':'style_id','rtl':'retail_price',
                  'season':'priority'},axis='columns',inplace=True)

# Standardize values
clean_values(df_new_website)

In [22]:
df_new_website.columns.values

array(['style_id', 'collection', 'priority', 'retail_price', 'gender',
       'case_size', 'case_material', 'dial_color', 'band_color',
       'band_material', 'clasp', 'movement', 'water_resistance',
       'country_of_origin', 'warranty'], dtype=object)

In [23]:
df_new_website.head(1)

,style_id,collection,priority,retail_price,gender,case_size,case_material,dial_color,band_color,band_material,clasp,movement,water_resistance,country_of_origin,warranty
0,es5206,scarlette,2022h3,140,ladies,32 mm,stainless steel,brown,gold,stainless steel,fold-over,quartz,5 atm,japan - jp,2 year limited


### Proposal

This dataset contains the planned order by XYZ for the new watch models.

In [24]:
df_new_plan = pd.read_excel('../data/raw_datasets/Plan Order 2022H3.xlsx',header=1)

In [25]:
df_new_plan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SKU         38 non-null     object 
 1   PLATFORM    38 non-null     object 
 2   MOVEMENT    38 non-null     object 
 3   MSRP        38 non-null     float64
 4   CASE        38 non-null     object 
 5   BAND        38 non-null     object 
 6   CASE SIZE   38 non-null     object 
 7   LUG WIDTH   38 non-null     object 
 8   GENDER      38 non-null     object 
 9   STATUS      38 non-null     object 
 10  Plan Order  38 non-null     float64
dtypes: float64(2), object(9)
memory usage: 3.6+ KB


In [26]:
# Standardize column names
snake_case(df_new_plan)

# Rename columns
df_new_plan.rename({'sku':'style_id','platform':'collection','msrp':'retail_price','case':'color'},axis='columns',inplace=True)

# Drop null row (not needed)
df_new_plan.dropna(subset='style_id', inplace=True)

# Select desired columns
df_new_plan = df_new_plan[['style_id', 'collection', 'retail_price','plan_order','color']]

# Standardize values
clean_values(df_new_plan)

### New Models Q3

This dataset contains the new models that will debut in Q3 2022. This data was not used for the project.

In [27]:
# df_q3_new = pd.read_excel('../data/raw_datasets/extra data/new models H3.xlsx',header = 1)

# # Drop empty rows (NaN)
# df_q3_new.dropna(how = 'all',inplace=True)

# # Drop unnecessary columns
# df_q3_new.drop(columns=['STATUS'],inplace=True)

# # Standardize column names
# df_q3_new.columns = df_q3_new.columns.astype(str).str.strip().str.lower().str.replace(' ','_')
# df_q3_new.columns = ['style_id', 'collection', 'movement_type', 'retail_price', 'color', 'band_color', 'case',
#        'lug_width', 'gender']

# # Standardize values
# clean_values(df_q3_new)

### New Models Q4

This dataset contains the new models that will debut in Q4 2022. This data was not used for the project.

In [28]:
# df_q4_new = pd.read_excel('../data/new models H4.xlsx')

# # Drop empty rows (NaN)
# df_q4_new.dropna(how = 'all',inplace=True)

# # Drop unnecessary columns
# df_q4_new.drop(columns=['Unnamed: 9'],inplace=True)

# # Infer column names
# df_q4_new.columns = ['style_id', 'collection', 'movement_type', 'retail_price', 'color', 'band_color', 'case',
#        'lug_width', 'gender']

# # Identify and deal with null values
# # price - will deal with later
# # lug width
# df_q4_new['lug_width'] = df_q4_new['lug_width'].fillna('No lug width reported')

# # Standardize values
# df_q4_new['gender'] = clean(df_q4_new['gender'])
# df_q4_new['collection'] = clean(df_q4_new['collection'])
# df_q4_new['movement_type'] = clean(df_q4_new['movement_type'])
# df_q4_new['color'] = clean(df_q4_new['color'])
# df_q4_new['band_color'] = clean(df_q4_new['band_color'])
# df_q4_new['gender'] = clean(df_q4_new['gender'])

### Holiday items - new items Q4

This dataset contains the new models that will debut in Q4 2022. It was determined to be a repeat of New Models Q4.

In [29]:
# df_holiday_new = pd.read_excel('../data/HOLIDAY NEW ITEMS 05-04-2022.xlsx')

# # Drop unnecessary columns
# df_holiday_new.drop(columns=['DESCRIPTION','SEASON'],inplace=True)

# # Standardize column names
# df_holiday_new.columns = df_holiday_new.columns.astype(str).str.strip().str.lower().str.replace(' ','_').str.replace('u_','').str.replace('ws_','')
# df_holiday_new.columns = ['style_id', 'collection', 'gender', 'wholesale_price', 'retail_price', 'movement_type',
#        'color', 'band_color', 'case']

# # deal with retail_price later

# df_holiday_new.info()

# # save to csv
# df_holiday_new.to_csv('../cleaned_datasets//holiday_q4_new_models.csv',index=False)

## Merging Datasets

### Merge old/current models from watch flow chart and website

In [30]:
df_models = pd.merge(left = df_watch_chart, right = df_website, how = 'outer', on = 'style_id', sort=True)
df_models = sort_columns(df_models) # sort datetime columns and move to the end

#### Validation of data

In [31]:
df_models.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1724 entries, 0 to 1723
Columns: 118 entries, style_id to 2022-12
dtypes: float64(101), object(17)
memory usage: 1.6+ MB


In [32]:
# Return the non-date columns of the data frame (i.e. no '2022-01' with sales data)
return_nondate_col(df_models).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1724 entries, 0 to 1723
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   style_id           1724 non-null   object 
 1   retail_price_x     1588 non-null   float64
 2   collection_x       1588 non-null   object 
 3   case               1588 non-null   object 
 4   gender_x           1588 non-null   object 
 5   status             1588 non-null   object 
 6   gender_y           1195 non-null   object 
 7   collection_y       1195 non-null   object 
 8   size               1195 non-null   float64
 9   wholesale_price    1195 non-null   float64
 10  retail_price_y     1195 non-null   float64
 11  color              1195 non-null   object 
 12  country_of_origin  1195 non-null   object 
 13  warranty           1195 non-null   object 
 14  band_color         1195 non-null   object 
 15  band_material      1195 non-null   object 
 16  case_material      1195 

In [33]:
# check the status of models where there was no match on the website
df_models[df_models['retail_price_y'].isna()]['status'].value_counts()

discd                 478
no status reported     50
2018h3                  1
Name: status, dtype: int64

In [34]:
# check the last two years of sales of model where there was no match on the website
return_date_col(df_models)[df_models['retail_price_y'].isna()].iloc[:,-25:-1].sum().sum()

0.0

In [35]:
df_models[df_models['retail_price_y'].isna() & (df_models['status'] != 'discd') & (df_models['status'] != 'no status reported')]

,style_id,retail_price_x,collection_x,case,gender_x,status,gender_y,collection_y,size,wholesale_price,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
1210,fs5477,165.0,rutherford,38mm,mens,2018h3,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


It seems that the this model is now discontinued (had no sales in the last two years) and the other watches had no sales in 2021. The missing information from the website shouldn't have a too huge of an impact, but this could be something explored later.

In [36]:
# check that status is only past watches
df_models['status'].value_counts()
# it is not obvious that the "No status reported" wawtches are old, current, or new models

discd                 690
no status reported    330
2019h4                 68
2020h1                 60
2020h3                 56
2020h2                 49
2019h3                 49
2021h3                 46
2021h4                 45
2019h2                 38
2022h1                 31
2021h1                 29
2021h2                 27
2018h4                 25
2019h1                 21
2018h3                  8
2020h4                  6
2022h2                  6
2018                    3
0                       1
Name: status, dtype: int64

#### Check where the retail prices differ between datasets

In [37]:
col = 'retail_price_x'
discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
discrepancy

,style_id,retail_price_x,retail_price_y
0,am4141,129.0,110.0
9,am4532,125.0,149.0
17,bq1010,125.0,129.0
18,bq1130,135.0,139.0
19,bq3115,135.0,139.0
...,...,...,...
1713,me3208,219.0,240.0
1714,me3209,249.0,260.0
1715,me3210,249.0,250.0
1716,me3211,189.0,210.0


In [38]:
# This is a tough decision... however for now, let's replace the prices with the website retail prices
# Rationale: If we use the price in the model to predict sales, the new model prices will not be scaled to the
# previous model prices

consolidate_columns(df_models, col,col[:-1]+'y',col[:-2])

#### Check where the Collection differ between datasets

In [39]:
col = 'collection_x'
collection_discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
collection_discrepancy

,style_id,collection_x,collection_y
0,am4141,colleague,serena
5,am4508,colleague,serena
17,bq1010,madeline,rhett
18,bq1130,madeline,flynn
19,bq3115,madeline,suitor
...,...,...,...
1697,me3189,carlie mini,carlie mini me
1698,me3190,fb,fb - 01 automatic
1699,me3191,fb-01,fb - 01 automatic
1700,me3195,neutra,neutra automatic


Lots of errors or small differences in naming on the XYZ dataset. Will totally replace the XYZ collections with those in the website.

In [40]:
consolidate_columns(df_models,'collection_y','collection_x','collection')

In [41]:
# Standardize collection names
df_models['collection'] = simplify_collections(df_models['collection'])

Num of collections, before: 205
Num of collections, after: 186


#### Check where Gender differs between datasets

In [42]:
col = 'gender_x'
gender_discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
gender_discrepancy

,style_id,gender_x,gender_y
17,bq1010,ladies,mens
18,bq1130,ladies,mens
26,bq3334,men,ladies
27,bq3407,men,ladies
28,bq3422,men,ladies
...,...,...,...
1711,me3206,gents,mens
1712,me3207,gents,mens
1713,me3208,gents,mens
1714,me3209,gents,mens


In [43]:
gender_discrepancy_list = list(set(zip(gender_discrepancy['gender_x'],gender_discrepancy['gender_y'])))
gender_discrepancy_list

[('men', 'mens'),
 ('unisex', 'mens'),
 ('gents', 'mens'),
 ('men', 'ladies'),
 ('ladies', 'mens')]

In [44]:
collection_and_gender_discrepancy = gender_discrepancy.merge(collection_discrepancy,left_index = True, right_index=True)
collection_and_gender_discrepancy

,style_id_x,gender_x,gender_y,style_id_y,collection_x,collection_y
17,bq1010,ladies,mens,bq1010,madeline,rhett
18,bq1130,ladies,mens,bq1130,madeline,flynn
26,bq3334,men,ladies,bq3334,machine,suitor mini
27,bq3407,men,ladies,bq3407,machine,suitor
28,bq3422,men,ladies,bq3422,grant,karli
...,...,...,...,...,...,...
1522,fs5929,gents,mens,fs5929,fb-01,fb - 01
1569,le1132,gents,mens,le1132,retro,retro pilot
1658,me3140,men,mens,me3140,grant sport automatic,grant sport
1667,me3154,men,mens,me3154,townsman,48mm townsman


In [45]:
gender_only_discrepancy = gender_discrepancy.drop(index = collection_and_gender_discrepancy.index)
gender_only_discrepancy_list = list(set(zip(gender_only_discrepancy['gender_x'],gender_only_discrepancy['gender_y'])))
gender_only_discrepancy_list

[('unisex', 'mens'), ('gents', 'mens'), ('men', 'mens'), ('ladies', 'mens')]

In [46]:
gender_only_discrepancy[gender_only_discrepancy['gender_x'] == 'ladies']

,style_id,gender_x,gender_y
983,fs5068ie,ladies,mens


In [47]:
df_models[df_models['style_id'] == 'fs5068ie'] # this seems to be a mistake

,style_id,case,gender_x,status,gender_y,size,wholesale_price,color,country_of_origin,warranty,...,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,retail_price,collection
983,fs5068ie,44mm,ladies,discd,mens,44.0,72.5,blue,japan - jp,2 - year international limited warranty,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145.0,grant


Upon inspection, it looks to be ok to replace all conflicts with the gender from the website.

In [48]:
consolidate_columns(df_models,'gender_y','gender_x','gender')

In [49]:
# Check if any groups can be combined
df_models['gender'].value_counts()

ladies    902
mens      571
men       250
unisex      1
Name: gender, dtype: int64

In [50]:
df_models['gender'] = df_models['gender'].replace('men','mens')

In [51]:
# Seems safe to fill in gender with mens (example is in blake collection)
df_models['gender'] = df_models['gender'].replace('unisex','mens')

In [52]:
df_models['gender'].value_counts()

ladies    902
mens      822
Name: gender, dtype: int64

#### Check where Case/Size differs between datasets

In [53]:
case_discrepancy = df_models[(df_models['case'] != df_models['size'])][['style_id','case','size','status']].dropna()
case_discrepancy

,style_id,case,size,status
0,am4141,28mm,28.0,no status reported
1,am4183,28mm,28.0,discd
5,am4508,28mm,28.0,discd
9,am4532,cecile,40.0,discd
17,bq1010,42mm,42.0,no status reported
...,...,...,...,...
1714,me3209,44mm,44.0,2021h4
1715,me3210,44mm,44.0,2021h4
1716,me3211,34mm,34.0,2021h4
1717,me3212,34mm,34.0,2021h4


In [54]:
len([38., 44., 45., 35., 34., 34., 32., 32., 32., 32., 32., 27., 36.,
       36., 36., 38., 38., 29., 29., 29., 35., 35., 29., 36., 36., 38.,
       38., 38., 36., 36., 36., 28., 28., 28., 28., 28., 35., 34., 34.,
       34., 35., 35., 44., 41., 44., 44., 44., 44., 48., 48., 48., 48.,
       48., 44., 46., 46., 36., 27., 35.])

59

It seems that case is the size with the units. Let's try stripping the MM and converting to float. There is clearly one example where this will fail, so let's replace with size in that instance.

In [55]:
case_size = []
count_except = 0
for i in case_discrepancy.index:
    try:
        case_size.append(float(case_discrepancy.loc[i,'case'].strip('mm')))
    except:
        case_size.append(case_discrepancy.loc[i,'size'])
        count_except +=1
print(count_except)

10


In [56]:
case_discrepancy['case_size'] = case_size
case_discrepancy

,style_id,case,size,status,case_size
0,am4141,28mm,28.0,no status reported,28.0
1,am4183,28mm,28.0,discd,28.0
5,am4508,28mm,28.0,discd,28.0
9,am4532,cecile,40.0,discd,40.0
17,bq1010,42mm,42.0,no status reported,42.0
...,...,...,...,...,...
1714,me3209,44mm,44.0,2021h4,44.0
1715,me3210,44mm,44.0,2021h4,44.0
1716,me3211,34mm,34.0,2021h4,34.0
1717,me3212,34mm,34.0,2021h4,34.0


In [57]:
case_discrepancy_actual = case_discrepancy[(case_discrepancy['size'] != case_discrepancy['case_size'])]

There are bizarre discrepancies on both datasets. I will have to manually replace issues.

In [58]:
replace_case = dict(zip(['ce1102', 'ch2601ie', 'ch2993', 'es4245', 'es4287', 'es4289',
       'es4336set', 'es4337set', 'es4363', 'es4372', 'es4374', 'es4412',
       'es4419', 'es4420', 'es4421', 'es4425', 'es4426', 'es4429',
       'es4430', 'es4431', 'es4432', 'es4442', 'es4507', 'es4629',
       'es4663', 'es4755', 'es4756', 'es4757', 'es4767', 'es4813',
       'es4814', 'es4833', 'es4834', 'es4835', 'es4837', 'es4839',
       'es4861', 'es4862', 'es4863', 'es4864', 'es4888', 'es4889',
       'fs5132', 'fs5411', 'fs5625', 'fs5637', 'fs5638', 'fs5640',
       'fs5687', 'fs5688', 'fs5689', 'fs5690', 'fs5691', 'fs5695',
       'fs5696', 'fs5697', 'le1095', 'me3071', 'me3165'],
[38., 44., 45., 35., 34., 34., 32., 32., 32., 32., 32., 27., 36.,
       36., 36., 38., 38., 29., 29., 29., 35., 35., 29., 36., 36., 38.,
       38., 38., 36., 36., 36., 28., 28., 28., 28., 28., 35., 34., 34.,
       34., 35., 35., 44., 41., 44., 44., 44., 44., 48., 48., 48., 48.,
       48., 44., 46., 46., 36., 27., 35.]))

In [59]:
for key, value in replace_case.items():
    i = df_models[df_models.style_id == key].index
    case_discrepancy_actual.loc[i,'case_size'] = value

In [60]:
new_case_size = []
for i in df_models.index:
    if i in case_discrepancy_actual:
        x = case_discrepancy_actual.loc[i,'case_size']
    elif str(df_models.loc[i,'size']) != 'nan':
        x = df_models.loc[i,'size']
    else:
        try:
            x = float(df_models.loc[i,'case'].strip('mm'))
        except:
            x = np.nan
    new_case_size.append(x)
df_models['case_size'] = new_case_size

In [61]:
df_models[df_models['case_size'].isna()][['case','size','case_size']]

,case,size,case_size
3,cecile,NaN,NaN
6,cecile,NaN,NaN
7,cecile,NaN,NaN
8,cecile,NaN,NaN
10,cecile,NaN,NaN
...,...,...,...
1651,dean,NaN,NaN
1652,machine,NaN,NaN
1653,machine,NaN,NaN
1656,muse,NaN,NaN


There are lots of models where the case size is missing. Let's see if there is a good strategy to impute the case size.

In [62]:
df_models.groupby('collection')['case_size'].std().sort_values(ascending = False)[0:20]

collection
fb-02                11.710801
daisy3h              10.000000
izzy                  9.811558
forresterchrono       9.276014
carlieminiv-day       8.000000
garrett               6.936639
andyandaddisonset     6.000000
retropilot            5.656854
stella                5.220539
carliemini            5.032733
carlie                4.779468
fb-01                 4.697781
tailor                4.281373
comuter3h             4.140393
copeland              3.938928
jacqueline            3.770258
atwater               3.577709
comuter3hdate         3.113996
bronson               3.070598
georgia               3.042555
Name: case_size, dtype: float64

It is not obvious what the case size should be. For now, I will replace with "No case size reported"

In [63]:
df_models['case_size'] = df_models['case_size'].fillna('no case size reported')

In [64]:
df_models.drop(columns=['case','size'],inplace=True)

#### Fill empty sales with 0.

In [65]:
fill_num_col_zero(df_models)

#### Fill empty and 0 status with "No status reported"

In [66]:
df_models['status'] = df_models['status'].fillna("no status reported")

In [67]:
df_models['status'].replace(0,"no status reported",inplace=True)
df_models['status'].replace('0',"no status reported",inplace=True)

In [68]:
df_models['status'].value_counts()

discd                 690
no status reported    467
2019h4                 68
2020h1                 60
2020h3                 56
2020h2                 49
2019h3                 49
2021h3                 46
2021h4                 45
2019h2                 38
2022h1                 31
2021h1                 29
2021h2                 27
2018h4                 25
2019h1                 21
2018h3                  8
2020h4                  6
2022h2                  6
2018                    3
Name: status, dtype: int64

#### Recalculate wholesale price based on retail price

In [69]:
df_models['wholesale_price'] = df_models['retail_price']*0.5

#### Examine color

There are discrepancies based on abbreviations and different spellings. Let's standardize.

In [70]:
df_models['color'] = df_models['color'].fillna("no color reported")

df_models['color'] = simplify_color(df_models['color'])

df_models['color'].value_counts()

Num of colors, before: 31
Num of colors, after: 28


no color reported    529
black                253
silver               208
white                133
blue                 130
rose gold            113
gold                  80
mother of pearl       49
grey                  42
green                 39
brown                 28
cream                 17
pink                  16
multicolor            13
smoke                 10
skeleton              10
red                    8
champagne              8
gunmetal               7
two-tone               6
purple                 6
caramel                6
clear                  5
burgundy               2
tan                    2
turquoise              2
yellow                 1
nude                   1
Name: color, dtype: int64

#### Examine country of origin

In [71]:
df_models['country_of_origin'].value_counts()

japan - jp       1141
china - cn         49
thailand - th       5
Name: country_of_origin, dtype: int64

In [72]:
df_models['country_of_origin'] = df_models['country_of_origin'].fillna("no country reported")

#### Examine warranty

In [73]:
df_models['warranty'].value_counts()

2 - year international limited warranty    1192
1 - year international limited warranty       3
Name: warranty, dtype: int64

In [74]:
df_models['warranty'] = df_models['warranty'].fillna("no warranty reported")

#### Examine band color

In [75]:
df_models['band_color'].replace('multi','multicolor',inplace=True)

df_models['band_color'] = df_models['band_color'].fillna("no color reported")

In [76]:
df_models['band_color'].value_counts()

no color reported    535
brown                256
silver               198
black                183
rose gold            116
blue                  91
gold                  70
grey                  47
multicolor            41
two-tone              40
green                 30
pink                  29
white                 24
nude                  22
red                   12
purple                 7
gunmetal               5
tan                    3
cream                  3
burgundy               3
rose                   2
champagne              2
yellow                 2
tortoise               1
turquoise              1
orange                 1
Name: band_color, dtype: int64

#### Examine band material

In [77]:
df_models['band_material'] = df_models['band_material'].fillna("no material reported")

In [78]:
df_models['band_material'].value_counts()

stainless steel         561
no material reported    529
leather                 526
silicone                 50
ceramic                  27
mesh                      9
alloy                     5
plastic                   5
acetate                   3
nylon                     3
fabric                    3
polyurethane              2
canvas                    1
Name: band_material, dtype: int64

#### Examine case material

In [79]:
df_models['case_material'] = df_models['case_material'].fillna("no material reported")

In [80]:
df_models['case_material'].value_counts()

stainless steel         1137
no material reported     529
ceramic                   27
alloy                     12
resin                      9
nylon                      7
acetate                    1
polyurethane               1
carbon                     1
Name: case_material, dtype: int64

#### Examine clasp type

In [81]:
df_models['clasp_type'] = df_models['clasp_type'].fillna("no clasp type reported")

In [82]:
df_models['clasp_type'].value_counts()

no clasp type reported    830
buckle                    447
fold-over                 306
deployment                112
clasp                      13
613                         6
safety                      4
velcro                      4
tang                        2
Name: clasp_type, dtype: int64

#### Examine dial color

In [83]:
df_models['dial_color'] = df_models['dial_color'].fillna("no color reported")

df_models['dial_color'] = simplify_color(df_models['dial_color'])

Num of colors, before: 25
Num of colors, after: 25


In [84]:
df_models['dial_color'].value_counts()

no color reported    529
black                285
blue                 156
white                148
silver               148
rose gold             76
mother of pearl       62
green                 60
grey                  44
gold                  41
brown                 31
cream                 26
pink                  23
digital               20
red                   20
multicolor            13
purple                10
skeleton              10
champagne             10
tan                    4
turquoise              2
clear                  2
burgundy               2
yellow                 1
beige                  1
Name: dial_color, dtype: int64

#### Examine movement type

In [85]:
df_models['movement_type'] = df_models['movement_type'].fillna("no movement type reported")

In [86]:
df_models['movement_type'].value_counts()

quartz                       1074
no movement type reported     536
automatic                      73
digital                        18
mechanical                     15
solar                           8
Name: movement_type, dtype: int64

#### Examine water resistance

In [87]:
df_models['water_resistant'] = df_models['water_resistant'].fillna("no water resistance reported")
df_models['water_resistant'] = df_models['water_resistant'].replace('nan',"no water resistance reported")

In [88]:
df_models['water_resistant'].value_counts()

50m - 160ft - 5atm              932
no water resistance reported    530
30m - 100ft - 3atm              146
100m - 330ft - 10atm            116
Name: water_resistant, dtype: int64

#### Examine band material

In [89]:
df_models['max_cart_qty'] = df_models['max_cart_qty'].astype(int).replace(0,"no max quantity reported")

In [90]:
df_models['max_cart_qty'].value_counts()

10000                       848
no max quantity reported    529
100                         347
Name: max_cart_qty, dtype: int64

### Merge df_models and df_proposal

In [91]:
df_models = df_models.merge(df_proposal, how = 'outer', on = 'style_id')

In [92]:
df_models.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1759 entries, 0 to 1758
Columns: 130 entries, style_id to qty_xyz_cost
dtypes: float64(109), object(21)
memory usage: 1.8+ MB


In [93]:
# Print the duplicated columns
for col in return_nondate_col(df_models).columns:
    if '_x' in col[-2::]:
        print(col)

wholesale_price_x
retail_price_x
collection_x
gender_x


In [94]:
# Wholesale price and retail price shouldn't have any issues, so use the consolidate identical function
for col in ['wholesale_price_x','retail_price_x']:
    if '_x' in col[-2::]:
        consolidate_identical_columns(df_models,col,col[:-1]+'y',col[:-2])

/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:215: UserWarning: wholesale_price_x and wholesale_price_y do not match
  warnings.warn(f'{priority_col} and {secondary_col} do not match')
/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:215: UserWarning: retail_price_x and retail_price_y do not match
  warnings.warn(f'{priority_col} and {secondary_col} do not match')


In [95]:
# Collection names are a different story...
# Simplify the collection names
df_models['collection_y'] = simplify_collections(df_models['collection_y'])

# Check for discrepancies in collection
df_models[df_models.collection_x != df_models.collection_y][['style_id','collection_x','collection_y','status']].dropna()

Num of collections, before: 55
Num of collections, after: 48


,style_id,collection_x,collection_y,status
522,es4432,carlie,carliemini,2018h3
523,es4433,carlie,carliemini,no status reported
562,es4502,carlie,carliemini,2018h4
910,es5176set,andyandaddisonset,addison,2022h1
1180,fs5439,minimalist,minimalist3h,no status reported
1184,fs5447,minimalist,minimalist-mono,no status reported
1193,fs5459,colorundertones,minimalist3h,no status reported
1223,fs5503,neutra,neutrachrono,2019h1
1250,fs5551,minimalist,minimalist3h,2019h2
1424,fs5822,everett,everett3h,no status reported


In [96]:
# Replace new collection names with the older ones - will make modeling more consistent
consolidate_columns(df_models,'collection_x','collection_y','collection')

In [97]:
# Check for discrepancies in collection
df_models[df_models.gender_x != df_models.gender_y][['style_id','gender_x','gender_y','status']].dropna()

,style_id,gender_x,gender_y,status
43,ce1107,ladies,women,no status reported
44,ce1108,ladies,women,no status reported
47,ce1111,ladies,women,2021h3
48,ce1112,ladies,women,2021h3
49,ce1113,ladies,women,2021h3
...,...,...,...,...
1713,me3208,mens,men,2021h4
1715,me3210,mens,men,2021h4
1716,me3211,ladies,women,2021h4
1719,me3214,ladies,women,2022h1


In [98]:
# Replace new genders with the older ones - will make modeling more consistent
consolidate_columns(df_models,'gender_x','gender_y','gender')

# There are some nulls, so replace men and women with mens and ladies
df_models.gender = df_models.gender.replace({'men':'mens','women':'ladies'})

In [99]:
# Identify how many watches did not have a match
# Priority indicates the "tier" of the watch
df_models[df_models['status'].isna()]['priority'].value_counts()

2022h3    34
2022h2     1
Name: priority, dtype: int64

In [100]:
df_models['priority'].value_counts()

c         98
b         93
2022h3    34
2022h2    32
2022h1    31
a         12
Name: priority, dtype: int64

Based on the priority of the watches that do not match, it seems that these are new models, hence why they were not on the watch flow chart or website. Let's see if they match up with the new models.

### Merge df_models and df_new_website

In [101]:
df_models = df_models.merge(df_new_website, how = 'left', on = 'style_id')

In [102]:
# Print the duplicated columns
dup_col = []
for col in return_nondate_col(df_models).columns:
    if '_x' in col[-2::]:
        dup_col.append(col)
dup_col

['country_of_origin_x',
 'warranty_x',
 'band_color_x',
 'band_material_x',
 'case_material_x',
 'dial_color_x',
 'case_size_x',
 'priority_x',
 'retail_price_x',
 'collection_x',
 'gender_x']

In [103]:
# Wholesale price and retail price shouldn't have any issues, so use the consolidate identical function
for col in ['country_of_origin_x',
 'warranty_x',
 'band_color_x',
 'band_material_x',
 'case_material_x',
 'dial_color_x',
 'case_size_x',
 'priority_x',
 'gender_x']:
    if '_x' in col[-2::]:
        consolidate_identical_columns(df_models,col,col[:-1]+'y',col[:-2])

/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in country_of_origin
  warnings.warn(f'There are null values in {new_col}')
/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in warranty
  warnings.warn(f'There are null values in {new_col}')
/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in band_color
  warnings.warn(f'There are null values in {new_col}')
/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in band_material
  warnings.warn(f'There are null values in {new_col}')
/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in case_material
  warnings.warn(f'There are null values in {new_col}')
/Users/vitoriamorenocosta/Desktop/DSI/p

#### Check where the retail price differ between datasets

In [104]:
col = 'retail_price_x'
discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
discrepancy

,style_id,retail_price_x,retail_price_y
1725,es5206,135.0,140.0
1739,es5234,145.0,150.0
1740,es5235,145.0,150.0
1744,fs5936,145.0,150.0
1747,fs5941,155.0,160.0


In [105]:
# replace older values with newer prices
consolidate_columns(df_models,'retail_price_y','retail_price_x','retail_price')

#### Check where the collection differ between datasets

In [106]:
col = 'collection_x'
discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
discrepancy

,style_id,collection_x,collection_y


In [107]:
# replace older values with newer prices
consolidate_columns(df_models,'collection_y','collection_x','collection')

In [108]:
# Reorganize columns
df_models = sort_columns(df_models)

### Merge df_models and df_new_plan

In [109]:
df_models = df_models.merge(df_new_plan, how = 'left', on = 'style_id')

In [110]:
df_models[['color_y', 'retail_price_y', 'collection_y']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1759 entries, 0 to 1758
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   color_y         33 non-null     object 
 1   retail_price_y  33 non-null     float64
 2   collection_y    33 non-null     object 
dtypes: float64(1), object(2)
memory usage: 55.0+ KB


In [111]:
# Print the duplicated columns
dup_col = []
for col in return_nondate_col(df_models).columns:
    if '_x' in col[-2::]:
        dup_col.append(col)
dup_col

['color_x', 'retail_price_x', 'collection_x']

In [112]:
# Wholesale price and retail price shouldn't have any issues, so use the consolidate identical function
for col in ['color_x']:
    if '_x' in col[-2::]:
        consolidate_identical_columns(df_models,col,col[:-1]+'y',col[:-2])

/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in color
  warnings.warn(f'There are null values in {new_col}')


#### Check where the retail price differ between datasets

In [113]:
col = 'retail_price_x'
discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
discrepancy

,style_id,retail_price_x,retail_price_y
1725,es5206,140.0,135.0
1739,es5234,150.0,145.0
1740,es5235,150.0,145.0
1744,fs5936,150.0,145.0
1747,fs5941,160.0,155.0


In [114]:
# keep original prices from the webiste
consolidate_columns(df_models,'retail_price_x','retail_price_y','retail_price')

#### Check where the collection differ between datasets

In [115]:
col = 'collection_x'
discrepancy = df_models[(df_models[col] != df_models[col[:-1]+'y'])][['style_id',col,col[:-1]+'y']].dropna()
discrepancy

,style_id,collection_x,collection_y
1726,es5212,carlie,carlie mini
1727,es5213,carlie,carlie mini
1728,es5214,carlie,carlie mini
1729,es5215,carlie,carlie mini
1751,fs5946,blue,fossil blue
1752,fs5947,blue,fossil blue
1753,fs5948,blue,fossil blue
1754,fs5949,blue,fossil blue
1755,fs5950,blue,fossil blue
1756,fs5951,blue,fossil blue


In [116]:
# keep original collection from the webiste
consolidate_columns(df_models,'collection_x','collection_y','collection')

In [117]:
# Reorganize columns
df_models = sort_columns(df_models)

In [118]:
return_nondate_col(df_models).columns

Index(['style_id', 'status', 'clasp_type', 'movement_type', 'water_resistant',
       'max_cart_qty', 'case', 'availability', 'qty_on_hand', 'qty_on_order',
       'qty_total_inv', 'qty_sales_order', 'qty_sold_last_6m', 'qty_avg/mo',
       'months_of_supply', 'xyz_cost', 'qty_xyz_cost', 'wholesale_price',
       'clasp', 'movement', 'water_resistance', 'country_of_origin',
       'warranty', 'band_color', 'band_material', 'case_material',
       'dial_color', 'case_size', 'priority', 'gender', 'plan_order', 'color',
       'retail_price', 'collection'],
      dtype='object')

In [119]:
# Find the non-null columns for the new models
new_non_null_col = []
for c in df_models:
    new_models = df_models[(df_models['status'].astype(str)=='nan')]
    if str(new_models.loc[new_models[new_models['warranty'].astype(str) != 'nan'].index[0] ,c]) != 'nan':
        new_non_null_col.append(c)

In [120]:
df_models[df_models['status'].isna() & df_models['warranty'].isna()][new_non_null_col]

,style_id,case,availability,qty_on_hand,qty_on_order,qty_total_inv,qty_sales_order,qty_sold_last_6m,qty_avg/mo,xyz_cost,...,band_material,case_material,dial_color,case_size,priority,gender,plan_order,color,retail_price,collection
1724,es5183set,28mm,available,0.0,0.0,0.0,40.0,0.0,0.0,49.00,...,NaN,NaN,NaN,NaN,2022h2,ladies,NaN,NaN,140.0,carlie
1758,le1156,42mm,delivery more than 6 weeks,0.0,0.0,0.0,0.0,0.0,0.0,68.25,...,NaN,NaN,NaN,NaN,2022h3,mens,NaN,NaN,195.0,blue


In [121]:
# Find the null columns for the new models where there should be values
null_col = []
for key,count in zip(new_models[new_non_null_col].isna().sum().keys(),new_models[new_non_null_col].isna().sum().values):
    if count > 0:
        null_col.append(key)
null_col

['clasp',
 'movement',
 'water_resistance',
 'country_of_origin',
 'warranty',
 'band_color',
 'band_material',
 'case_material',
 'dial_color',
 'case_size',
 'plan_order',
 'color']

In [122]:
# Manually fill in using data and website search
new_model_missing = {'es5183set':{'clasp':'deployment',
 'movement':'quartz',
 'water_resistance':'5 atm',
 'country_of_origin':'no country reported',
 'warranty':'2 year limited',
 'band_color':'gold',
 'band_material':'stainless steel',
 'case_material':'stainless steel',
 'dial_color':'silver',
 'case_size':'28',
 'plan_order':'nan',
 'color':'gold'}, 
                     'le1156':{'clasp':'buckle',
 'movement':'solar',
 'water_resistance':'10 atm',
 'country_of_origin':'no country reported',
 'warranty':'2 year limited',
 'band_color':'green',
 'band_material':'leather',
 'case_material':'stainless steel',
 'dial_color':'black',
 'case_size':'42',
 'plan_order':'nan',
 'color':'black'}}

for model, replacements in new_model_missing.items():
    i = df_models[df_models.style_id == model].index[0]
    for c, value in replacements.items():
        df_models.loc[i,c] = value

#### Inspect all non-date columns of the merged dataset

In [123]:
return_nondate_col(df_models).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1759 entries, 0 to 1758
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   style_id           1759 non-null   object 
 1   status             1724 non-null   object 
 2   clasp_type         1724 non-null   object 
 3   movement_type      1724 non-null   object 
 4   water_resistant    1724 non-null   object 
 5   max_cart_qty       1724 non-null   object 
 6   case               300 non-null    object 
 7   availability       300 non-null    object 
 8   qty_on_hand        300 non-null    float64
 9   qty_on_order       300 non-null    float64
 10  qty_total_inv      300 non-null    float64
 11  qty_sales_order    300 non-null    float64
 12  qty_sold_last_6m   300 non-null    float64
 13  qty_avg/mo         300 non-null    float64
 14  months_of_supply   216 non-null    float64
 15  xyz_cost           300 non-null    float64
 16  qty_xyz_cost       300 n

In [124]:
# make a list of columns that have null values to inspect further
null_col = []
for key, value in dict(return_nondate_col(df_models).isna().sum()).items():
    if value > 0:
        null_col.append(key)
set(null_col)

{'availability',
 'case',
 'clasp',
 'clasp_type',
 'max_cart_qty',
 'months_of_supply',
 'movement',
 'movement_type',
 'plan_order',
 'priority',
 'qty_avg/mo',
 'qty_on_hand',
 'qty_on_order',
 'qty_sales_order',
 'qty_sold_last_6m',
 'qty_total_inv',
 'qty_xyz_cost',
 'status',
 'water_resistance',
 'water_resistant',
 'xyz_cost'}

In [125]:
# Consolidate clasp
consolidate_identical_columns(df_models,'clasp_type','clasp','clasp_type_new')

In [126]:
# Compare, replace values, and consolidate water resistance
df_models.water_resistant.replace({'50m - 160ft - 5atm':'5 atm', '30m - 100ft - 3atm':'3 atm', '100m - 330ft - 10atm':'10 atm'},inplace=True)
consolidate_identical_columns(df_models,'water_resistant','water_resistance','water_resistance_new')

In [127]:
# Consolidate movement
consolidate_identical_columns(df_models,'movement_type','movement','movement_type_new')

In [128]:
# Consolidate status & priority ( status should be the primary, and keep a priority column)
df_models['proposal_priority'] = df_models.priority
consolidate_columns(df_models, 'status','priority','status_new')

In [129]:
# Clean case size (remove mm) and delete case
df_models.case_size = df_models.case_size.astype(str).str.replace(' mm','').replace('no case size reported',np.nan).astype(float).fillna('no case size reported')
df_models.drop(columns='case',inplace=True)

In [130]:
# Consolidate qty_sales_order and plan order
consolidate_columns(df_models,'plan_order','qty_sales_order','planned_order')

/Users/vitoriamorenocosta/Desktop/DSI/projects/capstone/code/custom_functions.py:152: UserWarning: There are null values in planned_order
  warnings.warn(f'There are null values in {new_col}')


In [131]:
# make a list of columns that have null values to inspect further
null_col = []
for key, value in dict(return_nondate_col(df_models).isna().sum()).items():
    if value > 0:
        null_col.append(key)
set(null_col)

{'availability',
 'max_cart_qty',
 'months_of_supply',
 'planned_order',
 'proposal_priority',
 'qty_avg/mo',
 'qty_on_hand',
 'qty_on_order',
 'qty_sold_last_6m',
 'qty_total_inv',
 'qty_xyz_cost',
 'xyz_cost'}

In [132]:
# Inspect the value counts to make sure all consistent and fix any last issues

In [133]:
# Replace XYZ cost, qty_xyz_cost, and wholesale price based on new retail price
df_models.wholesale_price = df_models.retail_price*0.5
df_models.xyz_cost = df_models.wholesale_price*0.7
df_models.qty_xyz_cost = df_models.xyz_cost*df_models.planned_order

In [134]:
# Replace inconsistent warranty
df_models.warranty.replace({'2 year limited':'2 - year international limited warranty','1 year limited':'1 - year international limited warranty'},inplace=True)

In [135]:
# Replace inconsistent colors
df_models.band_color = simplify_color(df_models.band_color)

Num of colors, before: 29
Num of colors, after: 26


In [136]:
# Replace inconsistent colors
df_models.dial_color = simplify_color(df_models.dial_color)

Num of colors, before: 27
Num of colors, after: 25


In [137]:
# Replace inconsistent colors
df_models.color = simplify_color(df_models.color)

Num of colors, before: 35
Num of colors, after: 28


In [138]:
# Simplify collections
df_models.collection = simplify_collections(df_models.collection)

Num of collections, before: 187
Num of collections, after: 187


In [139]:
# Fix one-off missing collection
df_models.loc[df_models[df_models.style_id == 'bq3210'].index[0],'collection'] = 'blythe'

In [140]:
# Remove new from the columns names
for c in df_models.columns:
    if c[-4::] == '_new':
        df_models.rename({c:c[:-4]},axis='columns',inplace=True)

In [141]:
return_nondate_col(df_models).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1759 entries, 0 to 1758
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   style_id           1759 non-null   object 
 1   max_cart_qty       1724 non-null   object 
 2   availability       300 non-null    object 
 3   qty_on_hand        300 non-null    float64
 4   qty_on_order       300 non-null    float64
 5   qty_total_inv      300 non-null    float64
 6   qty_sold_last_6m   300 non-null    float64
 7   qty_avg/mo         300 non-null    float64
 8   months_of_supply   216 non-null    float64
 9   xyz_cost           1759 non-null   float64
 10  qty_xyz_cost       300 non-null    float64
 11  wholesale_price    1759 non-null   float64
 12  country_of_origin  1759 non-null   object 
 13  warranty           1759 non-null   object 
 14  band_color         1759 non-null   object 
 15  band_material      1759 non-null   object 
 16  case_material      1759 

In [142]:
# Reorganize df_models for saving
sort_columns(df_models).to_csv('../data/cleaned_datasets/df_models.csv',index=False)

In [145]:
return_nondate_col(sort_columns(df_models)).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1759 entries, 0 to 1758
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   style_id           1759 non-null   object 
 1   max_cart_qty       1724 non-null   object 
 2   availability       300 non-null    object 
 3   qty_on_hand        300 non-null    float64
 4   qty_on_order       300 non-null    float64
 5   qty_total_inv      300 non-null    float64
 6   qty_sold_last_6m   300 non-null    float64
 7   qty_avg/mo         300 non-null    float64
 8   months_of_supply   216 non-null    float64
 9   xyz_cost           1759 non-null   float64
 10  qty_xyz_cost       300 non-null    float64
 11  wholesale_price    1759 non-null   float64
 12  country_of_origin  1759 non-null   object 
 13  warranty           1759 non-null   object 
 14  band_color         1759 non-null   object 
 15  band_material      1759 non-null   object 
 16  case_material      1759 